In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
# TODO: FILL IN PARAMETERS
# Model Parameter
model = "resnet34"
model_short = "resnet"
MODEL_MODULE = resnet34
# Dataset parameter
dataset = "cifar10"
dataset_short = "cifar"
DATASET_ENUM = DatasetClassN.Cifar

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 4
STANDARDIZE = True

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [5]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, data_class=DATASET_ENUM, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=STANDARDIZE)

NON Members
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 15000
Batchsize: 1
Inputs:tensor([[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], [0.9608, 0.9255, 0.9294,  ..., 0.9922, 0.9882, 0.9961], [0.9333, 0.9451, 0.9569,  ..., 1.0000, 0.9922, 0.9961], ..., [0.9333, 0.9333, 0.9451,  ..., 0.9961, 0.9961, 1.0000], [0.9725, 0.9451, 0.9529,  ..., 1.0000, 1.0000, 1.0000], [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], [0.9608, 0.9255, 0.9333,  ..., 0.9961, 0.9961, 1.0000], [0.9373, 0.9490, 0.9647,  ..., 0.9961, 0.9961, 1.0000], ..., [0.9294, 0.9333, 0.9451,  ..., 0.9961, 0.9961, 1.0000], [0.9725, 0.9451, 0.9529,  ..., 1.0000, 1.0000, 1.0000], [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], [0.9608, 0.9333, 0.9451,  ..., 0.9961, 0.9961, 1.0000], [0.9373, 0.9490, 0.9647,  ..., 0.9961, 0.9922, 1.0000], ..., [0

In [6]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 3.8133e-01,  2.8821e+00, -1.2807e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],  [[-7.4577e-01,  1.1768e+00, -1.3453e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[-5.6401e-01,  5.9299e+00, -2.2507e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00]],  [[-1.0205e+00,  1.1300e+00,  4.7559e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[-6.9174e-01,  4.0412e-01,  1.1027e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00]],  [[ 1.0305e+00,  3.1774e-01, -6.4132e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1

### Attack Model

Simple Model

In [7]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [8]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [9]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [10]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, data_class=DATASET_ENUM, device=DEVICE, test_dataset=False, standardize=STANDARDIZE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3373, 0.3765, 0.3843,  ..., 0.2078, 0.2118, 0.2235], [0.4392, 0.5412, 0.5059,  ..., 0.2510, 0.2549, 0.2235], [0.4275, 0.4510, 0.4471,  ..., 0.2627, 0.2549, 0.2314], ..., [0.5373, 0.5647, 0.5529,  ..., 0.3490, 0.2745, 0.1765], [0.4157, 0.4353, 0.4549,  ..., 0.1608, 0.0941, 0.0745], [0.3882, 0.4000, 0.3961,  ..., 0.0627, 0.0627, 0.0627]],[[0.3765, 0.4314, 0.4314,  ..., 0.3412, 0.3333, 0.3490], [0.5529, 0.6784, 0.6275,  ..., 0.4039, 0.4157, 0.3804], [0.5059, 0.5569, 0.5765,  ..., 0.4235, 0.4118, 0.3843], ..., [0.6627, 0.7059, 0.7059,  ..., 0.3843, 0.2902, 0.1843], [0.4824, 0.5137, 0.5373,  ..., 0.1843, 0.1098, 0.0824], [0.4431, 0.4627, 0.4627,  ..., 0.0745, 0.0745, 0.0745]],[[0.1882, 0.2157, 0.2549,  ..., 0.4980, 0.4941, 0.5176], [0.1882, 0.2941, 0.2745,  ..., 0.5647, 0.5882, 0.5490], [0.3804, 0.2941, 0.2353,  ..., 0.5843, 0.5804, 0.5569], ..., [0.7765, 0.8314,

Attack Evaluation DL Sample

In [11]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 1.9950e-02,  4.9705e-01,  3.0792e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 9.0088e-01, -5.1437e-01, -6.1488e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 5.6940e-01,  4.1555e-01, -4.3127e-01,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 1.1227e+00, -8.2182e-02, -4.2400e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 5.1104e-01,  4.1613e-02,  2.7438e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],  [[ 3.9403e-01,  5.6719e-01, -8.2624e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0

### Train Attack model

In [12]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 30000
Batchsize: 64
Inputs:tensor([[[ 3.7554e-01,  9.3794e-01, -3.2199e-01,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[-2.1624e-01,  8.1324e-01,  6.1543e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],  [[-3.5451e-01, -5.7595e-02, -1.1686e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00]],  [[-1.0730e+00,  2.2037e+00,  1.1004e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[-1.1708e+00,  2.6409e+00,  2.1594e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+

### Evaluation

In [13]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 0.70 with Correct: 141 and Total: 200


0.705